In [ ]:
%%capture
!pip install ../input/rsnapkgs/timm-0.2.2-py3-none-any.whl
!pip install ../input/pytorch-transformers/pytorch_transformers-1.2.0-py3-none-any.whl
!cp ../input/gdcm-conda-install/gdcm.tar .
!tar -xvzf gdcm.tar
!conda install --offline ./gdcm/gdcm-2.8.9-py37h71b2a6d_0.tar.bz2
print("done")

In [ ]:
import sys
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import platform
import os
import gc
import glob
import gdcm
import pydicom
from PIL import Image
import timm
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import cv2
from collections import defaultdict, namedtuple
import sys
import torch
from torch.backends import cudnn
from torch.nn import DataParallel
from torch.utils.data import DataLoader

from tqdm import tqdm
import torch.distributed as dist
import platform
from torch.utils.data import Dataset
from torch.utils.data.sampler import Sampler
import albumentations as A
from albumentations.pytorch import ToTensor
from torch import nn
from torch.nn.modules.dropout import Dropout
from torch.nn.modules.linear import Linear
from torch.nn.modules.pooling import AdaptiveAvgPool2d
from pytorch_transformers.modeling_bert import BertConfig, BertEncoder
import torch.nn.functional as F
from torch import nn
PATH = '../input/rsnasubcheck'
sys.path.append(PATH)

PATH = '../input/rsnastr2020/rsnastr'
sys.path.append(PATH)
np. set_printoptions(suppress=True)
from training.tools.utils import ip_window
from training.tools.utils import get_logger
from training.tools.config import load_config, RSNA_CFG
from training.zoo import classifiers
from training.zoo.sequence import SpatialDropout, LSTMNet, TransformerNet
from training.tools.utils import RSNA_CFG as CFG
from consistency_check import clean_sub, check_consistency

#from training.zoo import classifiers
logger = get_logger('Kaggle', 'INFO') 


In [ ]:
class args:
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    batchsize = 1
    imgbatchsize = 128
    folds = [0,1,2]
    load_train = False
    do_full = True
    delta = False

In [ ]:
# cfg = load_config(args.config, defaults=RSNA_CFG)
def dict2class(cfg):
    cfg['device'] = args.device
    cfg = namedtuple('Struct', cfg.keys())(*cfg.values())
    return cfg
cfgimg = dict2class(load_config(f'{PATH}/configs/effnetb5_lr5e4_multi.json'))
cfgseq1 = dict2class(load_config(f'{PATH}/configs/b5_seq_lstm.json', defaults=RSNA_CFG))
cfgseq2 = dict2class(load_config(f'{PATH}/configs/b5_seq_transformer_1layer.json', defaults=RSNA_CFG))
cfgseq3 = dict2class(load_config(f'{PATH}/configs/b5_seq_transformer_2layer.json', defaults=RSNA_CFG))

#### DataLoader

In [ ]:
class RSNAImageSequenceDataset(Dataset):

    def __init__(self, 
                 datadf,
                 folddf,
                 mode="train"):
        self.mode = mode
        self.datadf = datadf
        self.folddf = folddf
        self.imgclasses = CFG['image_target_cols']
        self.studyclasses = CFG['exam_target_cols']

    def __len__(self):
        return len(self.folddf)

    def __getitem__(self, idx):
        # idx = 1
        studyidx = self.folddf.iloc[idx].StudyInstanceUID
        seriesidx = self.folddf.iloc[idx].SeriesInstanceUID
        studydf = self.datadf.query('StudyInstanceUID == @studyidx')\
                        .query('SeriesInstanceUID == @seriesidx')
        imgs = []
        imgnames = []
        imglabels = []
        if self.mode == 'train':
            studylabels = studydf[self.studyclasses].iloc[0].values
        for i, samp in studydf.reset_index().iterrows():
            if self.mode == 'train':
                imglabels.append(samp['pe_present_on_image'])
            sampkey, img_name = self.image_file(samp)
            try:
                dicom_object = pydicom.read_file(img_name)
                img = ip_window(dicom_object)
            except:
                img = np.zeros(shape=(512,512,3), dtype=np.uint8)
            imgs.append(img)
            imgnames.append(sampkey)
        imgs = np.stack(imgs)
        if self.mode == 'train':
            labels = torch.tensor(imglabels)
            return {'img_name': imgnames, 'image': imgs, 'imglabels': imglabels, 'studylabels': studylabels}
        return {'img_name': imgnames, 'image': imgs}
    
    def image_file(self, samp):
        sampkey = '/'.join([samp.StudyInstanceUID, samp.SeriesInstanceUID,samp.SOPInstanceUID])
        imgname = os.path.join('../input/rsna-str-pulmonary-embolism-detection', 
                                self.mode,
                                sampkey) + '.dcm'
        return sampkey, imgname

def collateimgfn(batch):
    
    maxlen = max([l['image'].shape[0] for l in batch])
    for b in batch:
        masklen = maxlen-len(b['image'])
        b['mask'] = np.ones((maxlen))
        b['mask'][:masklen] = 0
    
    outbatch = {'image': np.concatenate([b['image'] for b in batch], 0)}
    outbatch['mask'] = torch.tensor(np.vstack([np.expand_dims(b['mask'], 0) \
                                                for b in batch])).float()
    outbatch['img_name'] = []
    for b in batch:
        outbatch['img_name'] += b['img_name'] 
        
    if 'imglabels' in batch[0]:
        for b in batch:
            masklen = maxlen-len(b['image'])
            b['imglabelsout'] = np.ones((maxlen))*-1
            b['imglabelsout'][masklen:] = b['imglabels']
        outbatch['imglabels'] = torch.tensor(np.vstack([np.expand_dims(b['imglabelsout'], 0) \
                                                for b in batch])).long()
        outbatch['studylabels'] = torch.tensor(np.vstack([np.expand_dims(b['studylabels'], 0) \
                                                for b in batch])).long()
    return outbatch

def batchPredsfn(studypreds, imgpreds, imgnames):
    preds = []
    batchdf = pd.DataFrame([i.split('/') for i in imgnames], 
                           columns = ['StudyInstanceUID', 'SeriesInstanceUID','SOPInstanceUID'])
    studies = batchdf.StudyInstanceUID.unique().tolist()
    for study, row in zip(studies, studypreds):
        for col, pred in zip(CFG['exam_target_cols'], row):
            preds.append([f'{study}_{col}', pred])
    for nm, pred in zip(batchdf.SOPInstanceUID, imgpreds):
        preds.append([nm, pred])
    return preds

def aug_batch(imgs, mean_, std_, device):
    imgs = imgs.to(device).float()
    return (((imgs / 255) - mean_) / std_).permute(0, 3, 1, 2).half()

In [ ]:
!cp ../input/rsna-str-pulmonary-embolism-detection/sample_submission.csv submission.csv
if args.load_train:
    trndf = pd.read_csv('../input/rsna-str-pulmonary-embolism-detection/train.csv')
if os.path.exists('../input/rsna-str-pulmonary-embolism-detection/train') and not args.do_full:
    tstdf = pd.read_csv('../input/rsna-str-pulmonary-embolism-detection/test.csv').head(2000)
else:
    tstdf = pd.read_csv('../input/rsna-str-pulmonary-embolism-detection/test.csv')

keycols = ['StudyInstanceUID', 'SeriesInstanceUID']
DEBUG = (tstdf.shape[0]==146853)
DEBUG

In [ ]:
loaderargs = {'num_workers' : 4, 'pin_memory': False, 'drop_last': False, 'collate_fn' : collateimgfn}
if args.load_train:
    trndataset = RSNAImageSequenceDataset(mode = 'train', 
                                folddf = trndf[keycols].drop_duplicates().reset_index(drop=True), 
                                datadf = trndf)
    trnloader = DataLoader(trndataset, batch_size=args.batchsize, shuffle=False, **loaderargs)
    
tstdataset = RSNAImageSequenceDataset(mode = 'test', 
                                folddf = tstdf[keycols].drop_duplicates().reset_index(drop=True), 
                                datadf = tstdf)
tstloader = DataLoader(tstdataset, batch_size=args.batchsize, shuffle=False, **loaderargs)

In [ ]:
def load_image_model(wtsnm, conf, device = args.device):
    nclasses = len(conf.image_target_cols) + len(conf.exam_target_cols)
    model = classifiers.__dict__[conf.network](encoder=f"{conf.encoder}_infer",
                                                  nclasses = nclasses,
                                                 infer = True)
    checkpoint = torch.load(wtsnm, map_location=torch.device(device))
    model.load_state_dict(checkpoint['state_dict'])
    model = model.half().to(device)
    model = model.eval()
    return model

def load_exam_model(wtsnm, cfg, embed_size = 2048, device = args.device):
    model = LSTMNet(embed_size, 
                       nimgclasses = len(cfg.image_target_cols), 
                       nstudyclasses = len(cfg.exam_target_cols),
                       LSTM_UNITS=cfg.lstm_units, 
                       DO = cfg.dropout)
    checkpoint = torch.load(wtsnm, map_location=torch.device(device))
    model.load_state_dict(checkpoint)
    model = model.half().to(device)
    model = model.eval()
    return model

def load_exam_modelx(wtsnm, cfg, device = args.device):
    model = TransformerNet(cfg)
    checkpoint = torch.load(wtsnm, map_location=torch.device(device))
    model.load_state_dict(checkpoint)
    model = model.half().to(device)
    model = model.eval()
    return model

In [ ]:
# Load image model
imgmodels = {}
for fold in args.folds:
    wtsnm = f'../input/rsnastr2020weights/classifier__RSNAClassifier_tf_efficientnet_b5_ns_04d_fold{fold}_img512_accum1___best'
    logger.info(f'Load image model : {wtsnm}')
    imgmodels[fold] = load_image_model(wtsnm, conf = cfgimg, device = args.device)

# Load image model
exammodels = defaultdict(list)
embed_size = 2048
for fold in args.folds:
    wtsnm = f'../input/rsnastr2020weights/exam_lstm_classifier__RSNAClassifier_tf_efficientnet_b5_ns_04d_fold{fold}_img512_accum1___best__all_size512_hidden512_best.bin'
    logger.info(f'Load lstm  exam model : {wtsnm}')
    exammodels[fold].append( load_exam_model(wtsnm, cfgseq1, embed_size = embed_size, device = args.device) )
    
# Load exam model
exammodelsx = defaultdict(list)
for fold in args.folds:
    for cfg in [cfgseq2, cfgseq3]:
        wtsnm = '../input/rsnastr2020weights/exam_transformer_classifier__RSNAClassifier_tf_efficientnet_b5_ns_04d_'
        wtsnm += f'fold{fold}_img512_accum1___best__all_size512_nlayers{cfg.nlayers}_intermediate{cfg.intermediate_size}_hidden{cfg.hidden_size}_best.bin'
        logger.info(f'Load xfrmr exam model : {wtsnm}')
        exammodelsx[fold].append( load_exam_modelx(wtsnm, cfg, device = args.device) )      

In [ ]:
subdf =  pd.read_csv('../input/rsna-str-pulmonary-embolism-detection/sample_submission.csv')
subdf = subdf.set_index('id')
mean_ =  torch.tensor(cfgimg.normalize['mean'])[None, None, None, :].to(args.device)
std_  =  torch.tensor(cfgimg.normalize['std'])[None, None, None, :].to(args.device)
subdf.label = 0.49999 # A lot of special consistency conditions for exactly 0.5 prediction

In [ ]:
pbar = tqdm(enumerate(tstloader), total = 1 + len(tstdataset)//tstloader.batch_size, ncols=0)
for step, batch in pbar:
    try:
        # Add in a loop here for dataloader
        emb = defaultdict(list)
        studypredsls = []
        imgpredsls = []
        embbatch = {}
        imgs = torch.tensor(batch["image"])
        imgnames = batch['img_name']
        mask = batch['mask'].half().to(args.device)
        # Split the batch into chunks and infer all the images
        for imgchunk in torch.split(imgs, args.imgbatchsize, dim=0):
            imgchunk = aug_batch(imgchunk, mean_, std_, args.device)
            imgchunk = imgchunk.to(args.device)
            with torch.no_grad():
                for fold in args.folds:
                    emb[fold].append(imgmodels[fold](imgchunk))
        del imgchunk, imgs, batch
        # Line up the embedding in single 2d array
        for fold in args.folds:
            emb[fold] = torch.cat(emb[fold], 0)
            # Use the mask to populate in (batch, maxlen of images, emb dim)
            embbatch[fold] = torch.zeros(len(mask.flatten()), emb[fold].shape[-1]).half().to(args.device)
            embbatch[fold][mask.flatten()==1] = emb[fold]
            embbatch[fold] = embbatch[fold].reshape((*mask.shape, emb[fold].shape[-1]))
            del emb[fold]
            for mod in exammodels[fold]:
                with torch.no_grad():
                    studylogits, imglogits = mod(embbatch[fold], mask)
                # Unmask the images
                imglogits = imglogits.flatten()[(mask==1).flatten()]
                # Detach and create out sub
                studypredsls.append( torch.sigmoid(studylogits).detach().cpu().numpy())
                imgpredsls.append( torch.sigmoid(imglogits).detach().cpu().numpy())
            try:
                for mod in exammodelsx[fold]:
                    with torch.no_grad():
                        encoded_layers = mod.encoder(embbatch[fold], *mod.extended_mask(mask ))
                        imglogits = mod.img_linear_out(encoded_layers[-1]).squeeze()
                        studylogits = mod.study_linear_out(encoded_layers[-1][:, -1]).squeeze()
                    # Unmask the images
                    imglogits = imglogits.flatten()[(mask==1).flatten()]
                    # Detach and create out sub
                    studypredsls.append( torch.sigmoid(studylogits).detach().cpu().numpy())
                    imgpredsls.append( torch.sigmoid(imglogits).detach().cpu().numpy())
            except:
                logger.info(f'Transformer failed ...{e}')
                torch.cuda.empty_cache()
            
        # Bag the batch
        studypreds = sum(studypredsls)/len(studypredsls)
        imgpreds = sum(imgpredsls)/len(imgpredsls)
        subbatch = batchPredsfn(studypreds, imgpreds, imgnames)
        subbatch = pd.DataFrame(subbatch, columns = ['id', 'label'])
        subdf.loc[subbatch.id, 'label'] = subbatch.label.values
        del mask, embbatch, imglogits, studylogits
    except Exception as e:
        logger.info(f'Failed ...{e}')
    if step % 1 == 0:
        torch.cuda.empty_cache()
    if step % 5 == 0:
        gc.collect()
subdf = subdf.reset_index()
suborig = subdf.copy()

In [ ]:
tstdf = pd.read_csv('../input/rsna-str-pulmonary-embolism-detection/test.csv')
subdf = clean_sub(subdf, tstdf)

In [ ]:
errors = check_consistency(subdf, tstdf)
if errors.shape[0] == 0:
    subdf.to_csv('submission.csv', index = False)
else:
    print(errors.broken_rule.value_counts())

In [ ]:
print(subdf.shape)
subdf.head(10)

In [ ]:
((subdf.label - suborig.label)!=0).value_counts()

In [ ]:
(subdf.label - suborig.label)[(subdf.label - suborig.label)!=0].hist(bins = 100)